# Feature Construction & Splitting
[source](https://www.youtube.com/watch?v=ma-h30PoFms&list=PLKnIA16_Rmvbr7zKYQuBfsVkjoLcJgxHH&index=45)  <br>
Not a rocket science, basically you split the columns which contain more than one piece of information. or have mixed data like numerical and categorical...   
**More importantly:** it depends on coder's intuition, there is no rule or mathematical formula that tells us which columns to split and what to split from it

In [24]:
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [25]:
df = pd.read_csv('assets/Titanic-Dataset.csv')[['Age','Pclass','SibSp','Parch','Survived']]
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [26]:
df.dropna(inplace=True)# age has missing values so.

In [27]:
X = df.iloc[:,0:4]
y = df.iloc[:,-1]

In [28]:
np.mean(cross_val_score(LogisticRegression(),X,y,cv=20))*100

np.float64(69.33333333333333)

## Applying Feature Construction

In [29]:
X['Family_size'] = X['SibSp'] + X['Parch'] + 1

In [32]:
def family_type(num):
    if num == 1:
        return 0 #alone
    elif num > 1 and num <=4:
        return 1 # small family
    else:
        return 2 # large family

In [33]:
family_type(1)

0

In [34]:
X['Family_type'] = X['Family_size'].apply(family_type)

In [35]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size,Family_type
0,22.0,3,1,0,2,1
1,38.0,1,1,0,2,1
2,26.0,3,0,0,1,0
3,35.0,1,1,0,2,1
4,35.0,3,0,0,1,0


In [37]:
X = X.drop(columns=['SibSp','Parch','Family_size',])

In [38]:
np.mean(cross_val_score(LogisticRegression(),X,y,cv=20))*100

np.float64(70.03174603174602)

## Applying Feature Splitting  

[also_see_this_notebook](../lame/HandlingMixedData.ipynb)

In [72]:
df = pd.read_csv('assets/Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
df['Name'] # let's say that we want to separate the salutation from names

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [74]:
df['Title'] = df['Name'].str.split(',' , expand=True)[1].str.split('.',expand=True)[0]

In [75]:
df[['Title','Name']]

,Title,Name
0,Mr,"Braund, Mr. Owen Harris"
1,Mrs,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,Miss,"Heikkinen, Miss. Laina"
3,Mrs,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,Mr,"Allen, Mr. William Henry"
...,...,...
886,Rev,"Montvila, Rev. Juozas"
887,Miss,"Graham, Miss. Margaret Edith"
888,Miss,"Johnston, Miss. Catherine Helen ""Carrie"""
889,Mr,"Behr, Mr. Karl Howell"


In [76]:
(df.groupby(by='Title')['Survived'].mean()).sort_values(ascending=False) # another analysis discovered

Title
Lady            1.000000
Ms              1.000000
Sir             1.000000
Mme             1.000000
the Countess    1.000000
Mlle            1.000000
Mrs             0.792000
Miss            0.697802
Master          0.575000
Major           0.500000
Col             0.500000
Dr              0.428571
Mr              0.156673
Capt            0.000000
Jonkheer        0.000000
Don             0.000000
Rev             0.000000
Name: Survived, dtype: float64

In [100]:
df['Title'].unique()


array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer'], dtype=object)

In [101]:
df['is_Married'] = 0
df.loc[df['Title'] == ' Mrs', 'is_Married'] = 1

In [102]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,is_Married
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,0


In [87]:
df['Title'].value_counts()


Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
Ms                1
Mme               1
Don               1
Lady              1
Sir               1
Capt              1
the Countess      1
Jonkheer          1
Name: count, dtype: int64